In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("../alerts.db")
cursor = conn.cursor()

# Fetch incidents from the past 7 days
query = """
SELECT type, severity, location, status
FROM alerts
WHERE timestamp >= DATE('now', '-7 days')
ORDER BY timestamp DESC;
"""

cursor.execute(query)
incidents = cursor.fetchall()  # Fetch all rows

# Close the database connection
conn.close()


In [2]:
print(incidents)

[('fire', 'critical', 'Al Mushrif, Abu Dhabi', 'in progress'), ('assault', 'high', 'Al Raha Beach, Abu Dhabi', 'resolved'), ('assault', 'critical', 'Reem Island, Abu Dhabi', 'resolved'), ('fire', 'warning', 'Yas Mall, Abu Dhabi', 'in progress'), ('assault', 'critical', 'Corniche Road, Abu Dhabi', 'unresolved'), ('fire', 'warning', 'Khalidiya Mall, Abu Dhabi', 'in progress'), ('fire', 'critical', 'Hamdan Street, Abu Dhabi', 'resolved')]


In [3]:
# Format incidents into a readable string
incident_text = "\n".join(
    f"Date: Type: {row[0]}, Severity: {row[1]}, Location: {row[2]}, Status: {row[3]}"
    for row in incidents
)

# Create the prompt for Gen AI
prompt = f"""
You are an AI assistant generating a weekly incident report. Be straight forward and maintain a structure that can be placed into a pdf file. Do not talk to me. Only give me the text that would be put in the PDF.
Analyze the following incident data and summarize key insights:

{incident_text}

Generate a structured report with:
- Total incidents
- Most common incident type
- Severity distribution
- Locations with the highest incidents
- Open vs closed case summary
- Key observations & recommendations
"""


In [4]:
incident_text

'Date: Type: fire, Severity: critical, Location: Al Mushrif, Abu Dhabi, Status: in progress\nDate: Type: assault, Severity: high, Location: Al Raha Beach, Abu Dhabi, Status: resolved\nDate: Type: assault, Severity: critical, Location: Reem Island, Abu Dhabi, Status: resolved\nDate: Type: fire, Severity: warning, Location: Yas Mall, Abu Dhabi, Status: in progress\nDate: Type: assault, Severity: critical, Location: Corniche Road, Abu Dhabi, Status: unresolved\nDate: Type: fire, Severity: warning, Location: Khalidiya Mall, Abu Dhabi, Status: in progress\nDate: Type: fire, Severity: critical, Location: Hamdan Street, Abu Dhabi, Status: resolved'

In [5]:
from http.client import HTTPException
import os
from openai import OpenAI
from dotenv import load_dotenv
import re

load_dotenv()
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

LLM_MODEL = "meta/llama-3.1-70b-instruct"
SUMMARIZER_MODEL = "deepseek-ai/deepseek-r1"

def extract_post_think_content(text: str) -> str:
    """Extract content after </think> tag."""
    match = re.search(r'</think>\s*(.*?)$', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text


embedding_client = OpenAI(api_key=NVIDIA_API_KEY, base_url="https://integrate.api.nvidia.com/v1")
llm_client = OpenAI(api_key=NVIDIA_API_KEY, base_url="https://integrate.api.nvidia.com/v1")
try:
        messages = [
            {"role": "system", "content": "You are an AI assistant generating a weekly incident report."},
            {"role": "user", "content": prompt}
        ]
        response = llm_client.chat.completions.create(
            model=SUMMARIZER_MODEL,
            messages=messages,
            temperature=0.6,
            top_p=0.7,
            max_tokens=4096,
            stream=False
        )
        summary = response.choices[0].message.content.strip()
        processed_summary = extract_post_think_content(summary)
except Exception as e:
    # logger.error(f"Error generating summary: {e}")
    print("error")
    # raise HTTPException(status_code=500, detail="Error generating summary.")

In [6]:
_summary


NameError: name '_summary' is not defined

In [7]:
from datetime import datetime, timedelta

print(datetime.now() - timedelta(days=7))

2025-01-28 23:51:20.835009


In [8]:
%pip install reportlab



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.1 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image

def save_report_to_pdf(report_text, logo_path, filename="weekly_incident_report.pdf"):
    """Generate a beautiful PDF with logo and improved design."""
    # Create a document template
    document = SimpleDocTemplate(filename, pagesize=letter)
    elements = []

    # Add logo at the top of the document
    logo = Image(logo_path, width=100, height=50)  # Adjust size as needed
    elements.append(logo)
    elements.append(Spacer(1, 12))  # Space between logo and title

    # Add a title for the report
    title_style = ParagraphStyle(name='Title', fontName='Helvetica-Bold', fontSize=18, alignment=1, textColor=colors.blue)
    title = Paragraph("Weekly Incident Report", style=title_style)
    elements.append(title)
    elements.append(Spacer(1, 12))

    # Add a subtitle for the report
    subtitle_style = ParagraphStyle(name='Subtitle', fontName='Helvetica', fontSize=12, alignment=1, textColor=colors.black)
    subtitle = Paragraph(f"Report generated on {datetime.now().strftime('%Y-%m-%d')}", style=subtitle_style)
    elements.append(subtitle)
    elements.append(Spacer(1, 24))  # Space before the main content

    # Add the summary report text
    text_style = getSampleStyleSheet()['Normal']
    report_paragraph = Paragraph(report_text, style=text_style)
    elements.append(report_paragraph)
    elements.append(Spacer(1, 12))

    # Footer with additional information
    footer_style = ParagraphStyle(name='Footer', fontName='Helvetica-Oblique', fontSize=8, alignment=2, textColor=colors.gray)
    footer = Paragraph("This report is confidential and for internal use only.", style=footer_style)
    elements.append(footer)

    # Build the document
    document.build(elements)

    print(f"Report saved as {filename}")

# Usage
logo_path = 'path_to_your_logo.png'  # Replace with the actual path to your logo image
save_report_to_pdf(processed_summary, logo_path)


Report saved as weekly_incident_report.pdf
